 - ディジタルな正弦波を作成する
 - 正弦波の周波数を指定して「聞くことのできる」波を作る
 - waveモジュールを用いたwav出力
 - scipyを用いたwav出力
 - Audio関数による再生

In [1]:
import wave
import numpy as np
from scipy.io import wavfile
from IPython.display import Audio

In [2]:
OUT_WAVE_FILE = "out_wave.wav"
OUT_SCIPY_WAVE_FILE = "out_scipy.wav"

In [3]:
n_channel = 1                   # モノラル
bitdepth = 2                      # 量子化ビット数 16 bit (2 byte)
n_framerate = 16000     # 標本化周波数 (Hz)

In [4]:
freq = 1000                     # 正弦波の周波数 (Hz)
duration = 2                    # 音の継続時間 (sec)
amplitude = 8000                # 正弦波の振幅

T = 1.0 / n_framerate           # 標本化周期 (sec)

In [5]:
# 正弦波作成
time = np.arange(0, duration, T)  # 継続時間に等しい標本点の作成
sine_wave = amplitude * np.sin(2 * np.pi * freq * time)

In [6]:
# サンプル数
n_frames = len(sine_wave)

In [7]:
# bytesオブジェクトへの変換
sound_frames = sine_wave.astype(np.int16).tobytes()

In [8]:
# wavの書き込み (waveモジュール)
with wave.open(OUT_WAVE_FILE, "w") as sound:
    sound.setnchannels(n_channel)    # チャネル数
    sound.setsampwidth(bitdepth)     # 量子化ビット数 (byte!)
    sound.setframerate(n_framerate)  # 標本化周波数 (Hz)
    sound.setnframes(n_frames)       # チャネルあたりのサンプル数
    sound.writeframes(sound_frames)  # 音声データの書き込み

In [9]:
# wavの書き込み (scipyモジュール) -> お手軽！
sine_wave = sine_wave.astype(np.int16)  # 16bit整数に変換
wavfile.write(OUT_SCIPY_WAVE_FILE, n_framerate, sine_wave)

In [10]:
Audio(sine_wave, rate=n_framerate) # 再生